In [24]:
import numpy as np
import pyvisa 
import time
import csv
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

rm = pyvisa.ResourceManager()

k2400_heater = rm.open_resource('GPIB0::2::INSTR') #good
k2400_sample_current = rm.open_resource('GPIB0::1::INSTR') #good
k2000_pt = rm.open_resource('GPIB0::26::INSTR') #good
k2000_sample_voltage = rm.open_resource('GPIB0::3::INSTR') #good

def get_current(device):
    return float(device.query(':MEASure:curr?').split(',')[1])

def get_voltage(device):
    return float(device.query(':MEASure:voltage?'))

def get_fres(device):
    return float(device.query(':MEASure:FRES?'))

def set_current(device, value):
    device.write(f':sour:current {value}')
    device.write(':outp on')

def set_current_range(device, value):
    device.write(f':sour:curr:range {value}')

def get_current_range(device):
    return device.query(':sour:curr:rang?')

def temp_fres(weerstand):
    #lineair:
    #source: https://www.sterlingsensors.co.uk/pt100-resistance-table
    #weerstand (ohm) -- #temp (K)
    #100 -- 273.15 -->(x2,y2)
    #18.49 -- 73.15 --> (x1, y1)
    (x1, y1) = (18.49, 73.15)
    (x2, y2) = (31.32, 103.15)
    temp = y1 + (y2-y1)/(x2-x1) * (weerstand - x1)
    return temp

def get_temp(device):
    fpres = get_fres(device)
    temp = temp_fres(fpres)
    return temp

def calc_temp(device): #other code, for comparison
    resistance = get_fres(device)
    return ((resistance/100 - 1) * 255.86572669) + 273.15


def I_V_scan(I_vals, write_location = None, sleep_time = 0.5):
    max_I = max(I_vals)
    if max_I == 0:
        raise ValueError
    else:
        i = 0
        while 10**(-i) >= max_I:
            i +=1
        range_max = 10**(-i+1)
    set_current_range(k2400_sample_current, range_max)
    I_actuals = []
    V_vals = []
    for I in I_vals:
        set_current(k2400_sample_current, I)
        time.sleep(sleep_time)
        I_actuals.append(I)
        V_vals.append(get_voltage(k2000_sample_voltage))
    
    if write_location is not None:
        file = open(write_location, "at+")
        file.write("I = ")
        file.write(str(I_actuals))
        file.write('\n') 
        file.write("V = "+str(V_vals) + '\n')
        file.close()
    set_current(k2400_sample_current, 0)
    return [I_actuals, V_vals]


def R_T_scan(write_location, current):
    Temps = []
    Resistances = []
    highT = True
    set_current(k2400_sample_current, current)
    try:
        while True:
            temp = get_temp(k2000_pt)
            if temp < 150:
                highT = False
            volt = get_voltage(k2000_sample_voltage)
            amp = get_current(k2400_sample_current)
            print('measured')
            Temps.append(temp)
            Resistances.append(volt/amp)
            if highT:
                time.sleep(1)
            else:
                time.sleep(0.1)
    except KeyboardInterrupt:
        file = open(write_location, "at+")
        file.write('T = ' + str(Temps) + '\n')
        file.write('R = ' + str(Resistances) + '\n')
        file.close()
        



In [13]:
set_current(k2400_heater, 0.05)


In [28]:
calc_temp(k2000_pt)

108.46484056243798

In [ ]:
R_T_scan('Data/R_T_testing4.txt', .0001)

In [12]:
k2400_sample_current.write(':outp off')

11

In [21]:
set_current(k2400_sample_current, 0)

In [11]:
get_voltage(k2000_sample_voltage)

-3.79211124e-06

In [13]:
k2000_pt.query(':MEASure:FRES?')

'+2.19640358E+01\n'

In [3]:
V = I_V_scan([0.01,0.02,0.03,0.04,0.05,0.06], "Data/testl.txt")

In [22]:
namen = ["Data/test2.txt", "Data/test3.txt", "Data/test4.txt", "Data/test5.txt"]
for i in range(4):
    I_range = np.array(range(1,11))*10**(-i-1)
    I_V_scan(list(I_range), namen[i])

I_range

array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009, 0.001 ])

In [29]:
R_T_scan("Data/cooldown_10micro.txt", .00001)

measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured
measured


In [31]:
I_range = np.linspace(10**(-5), 1, 100)
I_V_scan(I_range, "Data/long_sweep-97.txt", sleep_time=0.1)


[[1e-05,
  0.01011090909090909,
  0.02021181818181818,
  0.030312727272727272,
  0.04041363636363637,
  0.05051454545454546,
  0.06061545454545455,
  0.07071636363636363,
  0.08081727272727272,
  0.09091818181818181,
  0.1010190909090909,
  0.11112,
  0.12122090909090909,
  0.1313218181818182,
  0.14142272727272728,
  0.15152363636363636,
  0.16162454545454547,
  0.17172545454545457,
  0.18182636363636365,
  0.19192727272727272,
  0.20202818181818183,
  0.21212909090909093,
  0.22223,
  0.2323309090909091,
  0.2424318181818182,
  0.2525327272727273,
  0.2626336363636364,
  0.27273454545454545,
  0.28283545454545456,
  0.29293636363636366,
  0.3030372727272727,
  0.3131381818181818,
  0.3232390909090909,
  0.33334,
  0.34344090909090913,
  0.3535418181818182,
  0.3636427272727273,
  0.3737436363636364,
  0.38384454545454544,
  0.39394545454545454,
  0.40404636363636365,
  0.41414727272727275,
  0.42424818181818186,
  0.4343490909090909,
  0.44445,
  0.4545509090909091,
  0.4646518181818

In [39]:
I_range_fine = np.linspace(0, 10**(-3), 400)
I_V_scan(I_range_fine, "Data/fine_sweep-104.txt", sleep_time=0.02)


[[0.0,
  2.506265664160401e-06,
  5.012531328320802e-06,
  7.518796992481203e-06,
  1.0025062656641604e-05,
  1.2531328320802006e-05,
  1.5037593984962406e-05,
  1.7543859649122806e-05,
  2.0050125313283208e-05,
  2.255639097744361e-05,
  2.506265664160401e-05,
  2.756892230576441e-05,
  3.007518796992481e-05,
  3.258145363408521e-05,
  3.508771929824561e-05,
  3.759398496240601e-05,
  4.0100250626566415e-05,
  4.260651629072682e-05,
  4.511278195488722e-05,
  4.761904761904762e-05,
  5.012531328320802e-05,
  5.263157894736842e-05,
  5.513784461152882e-05,
  5.764411027568922e-05,
  6.015037593984962e-05,
  6.265664160401002e-05,
  6.516290726817042e-05,
  6.766917293233083e-05,
  7.017543859649122e-05,
  7.268170426065163e-05,
  7.518796992481203e-05,
  7.769423558897244e-05,
  8.020050125313283e-05,
  8.270676691729323e-05,
  8.521303258145363e-05,
  8.771929824561403e-05,
  9.022556390977444e-05,
  9.273182957393483e-05,
  9.523809523809524e-05,
  9.774436090225564e-05,
  0.00010025